In [2]:
%load_ext rpy2.ipython

In [16]:
%%R
library("rjags")
mu.frac <- c(0.074, 0.32)
gamma.mean.dn <- c(20, 4.7)
beta.dn <- c(1,1)
gamma.mean.CC <- c(2.3, 1.00)
beta.CC <- c(4.0, 1000)
rho1 <- c(0.1, 0.5)
nu1 <- c(200, 100)
rho0 <- c(0.1, 0.5)
nu0 <- c(200, 100)
hyperpar <- as.array(rbind(gamma.mean.dn, beta.dn, gamma.mean.CC, beta.CC, rho1, nu1, rho0, nu0))
l <- 100
pi0 <- 0.94 # the fraction of non-risk genes

# ASC (Autism Sequencing Consortium) data
# The file name contains the sample size information
# The only relevant counts are dn.LoF and dn.mis3
data <- read.csv("TADA/data/ASC_2231trios_1333trans_1601cases_5397controls.csv", header=TRUE, as.is=TRUE)
ntrio <- 2231  # number of trios
ncase <- 1601  # number of cases
nctrl <- 5397  # number of controls
ntrans <- 1333 # number of subjects with transmission data
N <- list(dn=ntrio, ca=ntrans+ncase, cn=ntrans+nctrl)

# Running TADA
counts <- as.array(cbind(data$dn.LoF, data$case.LoF+data$trans.LoF, data$ctrl.LoF+data$ntrans.LoF, data$dn.mis3, data$case.mis3+data$trans.mis3,
 data$ctrl.mis3+data$ntrans.mis3))


count.mu.Data <- cbind(counts[, 1:3], data$mut.rate*mu.frac[1])



y <- as.numeric(counts[, 1])
murate = data$mut.rate*mu.frac[1]

Ncount <- N
NN <- length(y)

dataJags <- list(y = y,
              alpha0 = c(1, 1),
              NN = NN,
              T = c(NA, rep(NA, NN - 2), NA),
              murate = murate,
#              m1 = m1, m2 = m2,
              Ndn = N$dn)

jagsFile = "model {

           p1[1:2] ~ ddirch(alpha0[])


for (i in 1:NN){


            gamma.mean.dn[i, 1] <- 1
            beta.mean.dn[i, 1]  <- 100000
            
            gamma.mean0[i] ~ dbeta(13, 43)
            gamma.mean.dn[i, 2] <- 7 + 43*gamma.mean0[i]
            beta.mean.dn[i, 2] ~ dunif(0.2, 1.2)

            sizeI[i] <- gamma.mean.dn[i, T[i]]*beta.mean.dn[i, T[i]]
            probI[i] <- beta.mean.dn[i, T[i]]/(beta.mean.dn[i, T[i]] + 2*Ndn*murate[i])

           y[i] ~ dnegbin(probI[i], sizeI[i])

           T[i] ~ dcat(p1[])
       }

}
"



In [17]:
%%R


n.chains = 1
n.adapt = 100
mixture <- jags.model(textConnection(jagsFile),# inits = inits,
                              data = dataJags, n.chains = n.chains, n.adapt = n.adapt)

nUpdate = 1000
update(mixture, nUpdate)



Compiling model graph
   Resolving undeclared variables
   Allocating nodes
   Graph Size: 266454

Initializing model

  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100%
  |**************************************************| 100%


In [ ]:
%%R
n.iter = 1000
thin = 1
xem <- coda.samples(mixture, c('gamma.mean.dn', 'beta.mean.dn', 'T', 'p1'),
                    n.iter = n.iter, thin = thin)
##Each chain will produce a list


In [ ]:
%%R
a0 <- xem[[1]]

dim(a0)

In [16]:
!pwd

/usr/data/Documents/GitHub/TestR/HBproject/extTADA


In [17]:
#!gist -p test_different_prior_from_server.ipynb

In [18]:
!gist -u https://gist.github.com/8f7f13e8d497aa09b721 test_different_prior_from_server.ipynb

https://gist.github.com/8f7f13e8d497aa09b721
